In [1]:
def estimate_critical_flow_speed_all_in_month(cube, target_month, along_target, upper_layer_range=(0, 100), lower_layer_range=(100, 130)):
    import numpy as np
    import gsw
    from scipy.interpolate import interp1d

    results = []
    transects = cube.transect.values
    matching = [t for t in transects if str(target_month) in str(t)]
    if not matching:
        print(f"No transects found for month {target_month}")
        return []

    for transect in matching:
        ds = cube.sel(transect=transect)
        along = ds['along'].values
        idx = np.argmin(np.abs(along - along_target))

        T = ds['temperature'][:, idx].values
        S = ds['salinity'][:, idx].values
        depth = ds['depth'].values

        p = gsw.p_from_z(-depth, 51.5)
        rho = gsw.density.rho(S, T, p)

        # Masks for layers
        upper_mask = (depth >= upper_layer_range[0]) & (depth < upper_layer_range[1])
        lower_mask = (depth >= lower_layer_range[0]) & (depth < lower_layer_range[1])

        if np.sum(upper_mask) < 1 or np.sum(lower_mask) < 1:
            continue

        rho1 = np.nanmean(rho[upper_mask])
        rho2 = np.nanmean(rho[lower_mask])

        g = 9.81
        g_prime = g * (rho2 - rho1) / rho2
        h = lower_layer_range[1] - lower_layer_range[0]
        U = np.sqrt(g_prime * h**3)

        results.append({
            'transect': str(transect),
            "rho1": rho1,
            "rho2": rho2,
            "g'": g_prime,
            "U (m/s)": U,
            "h (m)": h
        })

    return results

results = estimate_critical_flow_speed_all_in_month(
    cube,
    target_month="202405",
    along_target=23500,
    upper_layer_range=(0, 100),
    lower_layer_range=(100, 130)
)

for r in results:
    print(r)

NameError: name 'cube' is not defined